In [1]:
from pyspark.sql import SparkSession


In [2]:

spark = SparkSession.builder \
    .appName('Ingest checkin table into bronze') \
    .master('spark://spark-master:7077') \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083")\
    .config("spark.hadoop.fs.s3a.access.key", 'minio') \
    .config("spark.hadoop.fs.s3a.secret.key", 'minio123') \
    .config("spark.hadoop.fs.s3a.endpoint", 'minio:9000')\
    .config("spark.hadoop.fs.s3a.path.style.access", "true")\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")\
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config('spark.sql.warehouse.dir', f's3a://lakehouse/')\
    .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4')\
    .enableHiveSupport()\
    .getOrCreate()

In [3]:
spark

In [4]:
t = spark.read \
    .format("parquet") \
    .option("header", "true") \
    .load("s3a://lakehouse/silver/customer/silver_cleaned_customers.parquet")

In [5]:
t.show()

+--------------------+--------------------+------------------------+--------------+--------------+
|         customer_id|  customer_unique_id|customer_zip_code_prefix| customer_city|customer_state|
+--------------------+--------------------+------------------------+--------------+--------------+
|801c36afa9f92d28f...|9fce80af3da5a57c4...|                   30575|belo horizonte|          MG\r|
|801c629f9d3579bdd...|9039fa4063ed33243...|                   60150|     fortaleza|          CE\r|
|801cf4cbead385894...|178037c9f75834915...|                   29295|   vargem alta|          ES\r|
|801d3adce37e5f15c...|5128cfed3714b48fe...|                    5265|     sao paulo|          SP\r|
|801e6d59e9665f5ed...|915219fa5f275ea30...|                    1224|     sao paulo|          SP\r|
|801fc27d9fbae58f4...|50db7e9b6d3b56e8e...|                   70382|      brasilia|          DF\r|
|80242677269637e36...|7df622c0386808fe5...|                   21863|rio de janeiro|          RJ\r|
|80242c91d

In [6]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS bronze")
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|   bronze|
|  default|
+---------+



In [8]:
spark.stop()

In [7]:
t.writeTo("bronze.restaurant_transform").using('delta').create()

AnalysisException: Table bronze.restaurant_transform already exists

In [8]:
t.write.format("delta").mode("overwrite").saveAsTable("bronze.restaurant_transform")
spark.sql("SHOW TABLES IN bronze").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|   bronze|restaurant_transform|      false|
+---------+--------------------+-----------+

